In [ ]:
%matplotlib inline

# Grid Generation on Climatology

As a first step in coordinate development, we'll work on sections of climatology from [WOA13](https://www.nodc.noaa.gov/OC5/woa13/). Because temperature is given in-situ, we first have to convert to potential temperature. Similar to [convert_WOA13](https://github.com/adcroft/convert_WOA13), we use the Python `gsw` package, which implements TEOS-10.

In [ ]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
from remapping import mom_remapping
import gsw
from scipy.linalg import solve_banded
from scipy.interpolate import interp1d
import m6toolbox

As an initial set of input files, we load up THREDDS URLs for monthly averaged climatologies from 2005-2012.

In [ ]:
url_format = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/{}/netcdf/A5B2/1.00/woa13_A5B2_{}{:02}_01v2.nc'

In [ ]:
url_format_025 = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/{}/netcdf/A5B2/0.25/woa13_A5B2_{}{:02}_04v2.nc'

In [ ]:
temp_urls = [url_format.format('temperature', 't', i+1) for i in range(12)]
salt_urls = [url_format.format('salinity', 's', i+1) for i in range(12)]

temp_urls_025 = [url_format_025.format('temperature', 't', i+1) for i in range(12)]
salt_urls_025 = [url_format_025.format('salinity', 's', i+1) for i in range(12)]

# Sections

Although our choice of coordinate should apply globally, we're particularly interested in a few troublesome spots, where there tend to always be problems, such as the Denmark Strait and the Sulu Sea. We may also care about dense overflows off Antarctica.

## Atlantic
We have an Atlantic section at around 25 degrees west, which should include a portion of Denmark Strait overflow. We'll start with state data from the 1 degree WOA13 dataset.

In [ ]:
temp = Dataset(temp_urls[0], 'r')
salt = Dataset(salt_urls[0], 'r')

In [ ]:
lat = temp.variables['lat'][:]
lon = temp.variables['lon'][:]
dep = temp.variables['depth'][:]

In [ ]:
lon_w = -25.5
#lon_i = (lon_s <= lon) & (lon <= lon_e)

t_sect = temp.variables['t_an'][0,:,:,lon==lon_w].squeeze()
s_sect = salt.variables['s_an'][0,:,:,lon==lon_w].squeeze()

We want to take an contiguous latitude section, so we'll remove Greenland and everything north of it, as well as Antarctica. There's also a weird masked column at around 35 which is probably an island...

In [ ]:
# empty columns are entirely masked
empty = np.sum(~t_sect.mask, axis=0) == 0
empty[169:] = True # mask above Greenland

In [ ]:
lat = lat[~empty]
t_sect = t_sect[:,~empty]
s_sect = s_sect[:,~empty]

Using TEOS-10, we can convert from practical salinity to absolute salinity, and from in-situ temperature to conservative temperature. From here, we can compute the locally-referenced density, and the potential density referenced to 2000m.

In [ ]:
sa_sect = np.empty_like(s_sect)
ct_sect = np.empty_like(t_sect)
rho_sect = np.empty_like(s_sect)
rhop_sect = np.empty_like(rho_sect)

for i in range(s_sect.shape[1]):
    sa_sect[:,i] = gsw.SA_from_SP(s_sect[:,i], dep, lon_w, lat[i])
    ct_sect[:,i] = gsw.CT_from_t(sa_sect[:,i], t_sect[:,i], dep)
    rho_sect[:,i] = gsw.rho(sa_sect[:,i], ct_sect[:,i], dep)
    rhop_sect[:,i] = gsw.rho(sa_sect[:,i], ct_sect[:,i], 2000)

Let's take a look at the two density sections, the locally-refenced density and the potential density referenced to 2000m.

In [ ]:
ax = plt.subplot(211)
plt.pcolormesh(lat, dep, rho_sect)
ax.invert_yaxis()
#plt.colorbar()

ax = plt.subplot(212)
plt.pcolormesh(lat, dep, rhop_sect)
ax.invert_yaxis()
#plt.colorbar()

# Observation Grid

We have observations at particular depth levels, which are given by masking the depth coordinate by the mask from a particular column of our temp/salt data. We assume these observations are at interfaces (because we have an observation at z=0, for example), and therefore we calculate layer averages from these so that we can perform remapping later. We also store the actual depth of each column (we don't have partial-cell topography available though) so that we can ensure regularity of our interpolated grid.

In [ ]:
# depths of all interfaces on which observations are present
gr_int = np.ma.array(np.tile(dep.reshape(-1, 1), (1, t_sect.shape[1])), mask=t_sect.mask)

# thicknesses of all layers between interfaces
gr_th  = np.diff(gr_int, axis=0)

# bottom interface at each column
topo = gr_int.max(axis=0).compressed()

In [ ]:
sa_lay = sa_sect[:-1,:] + np.diff(sa_sect, axis=0) / gr_th
ct_lay = ct_sect[:-1,:] + np.diff(ct_sect, axis=0) / gr_th

# Remapping

Now we can define a function that will remap from our source data to any target grid.

**Note:** *We might want to do this for temp/salt rather than conservative temp and absolute salt?*

In [ ]:
remap_cs = mom_remapping.Remapping_Cs()
remap_cs.remapping_scheme = 4 # PQM_IH4IH3
remap_cs.degree = 4

In [ ]:
def remap(h):
    """
    Remap from original climatological grid according to h
    """

    sa_remap = np.empty_like(h)
    ct_remap = np.empty_like(h)

    # remap by columns
    for i in range(h.shape[1]):
        # we need to make sure we deal with unmasking here,
        # otherwise we'll get the fill values for thickness
        # and salt/temp, which would be just a little weird
        sa_remap[:,i] = mom_remapping.remapping_core_h(gr_th[:,i].compressed(),
                                                       sa_lay[:,i].compressed(),
                                                       h[:,i], remap_cs)
        ct_remap[:,i] = mom_remapping.remapping_core_h(gr_th[:,i].compressed(),
                                                       ct_lay[:,i].compressed(),
                                                       h[:,i], remap_cs)
        
    return sa_remap, ct_remap

# Grid Generation

We can go about generating the grid using the Hofmeister et al. (2010) technique. This involves a vertical grid diffusion to optimise for buoyancy, shear, near-surface zooming and a background component. Then we're left with an isopycnal or neutral density curvature tendency term.

## Neutral Density Curvature

First, we'll define a function to calculate the neutral density curvature `ndc` at interfaces, which is where we need to calculate the tendency term.

In [ ]:
def ndc_int(z_int, sa_lay, ct_lay):
    """
    Calculate neutral density curvature between
    adjacent columns, given their (absolute) salinity, (conservative)
    temperature and physical positions (or pressure).
    
    z_int gives the current location of all model interfaces,
    and sa_lay and ct_lay give the absolute salinity and conservative
    temperature cell mean values between these interfaces.
    """
    
    def d(x):
        """
        Calculate diff and squeeze
        """
        
        return np.diff(x, axis=1).squeeze()
    
    # first, calculate layer thicknesses
    h = np.diff(z_int, axis=0)
    
    # use the layer thicknesses to interpolate interface values
    # using weighted average of the cell mean values on either side
    sa_int = (sa_lay[1:,:] * h[1:,:] + sa_lay[:-1,:] * h[:-1,:]) / (h[1:,:] + h[:-1,:])
    ct_int = (ct_lay[1:,:] * h[1:,:] + ct_lay[:-1,:] * h[:-1,:]) / (h[1:,:] + h[:-1,:])
    
    # drop top and bottom interfaces, since we won't calculate
    # the curvature at either of these places (those interfaces
    # can't move)
    z_int = z_int[1:-1]
    
    # for dealing with edges, extend data with a ghost column
    # to give a Neumann boundary condition (dC/dx = 0)
    int_gst = np.concatenate([ z_int[:,[0]],  z_int,  z_int[:,[-1]]], axis=1)
    sa_gst  = np.concatenate([sa_int[:,[0]], sa_int, sa_int[:,[-1]]], axis=1)
    ct_gst  = np.concatenate([ct_int[:,[0]], ct_int, ct_int[:,[-1]]], axis=1)
    
    # neutral density curvature *on interfaces*
    ndc = np.empty_like(z_int)

    # calculate for each (real) column
    for i in range(ndc.shape[1]):
        # calculate difference to column at left
        # first, we need the mean S, T and P between
        # the centre and left
        sa =  sa_gst[:,[i,i+1]]
        ct =  ct_gst[:,[i,i+1]]
        z  = int_gst[:,[i,i+1]]

        # now calculate the density, thermal expansion and
        # haline contraction at these mean values
        r, a, b = gsw.rho_alpha_beta(sa.mean(axis=1),
                                     ct.mean(axis=1),
                                      z.mean(axis=1))
        # use these to calculate the actual neutral density difference
        ndd_l = r * (b * d(sa) - a * d(ct))
        
        # use the same process as above
        # to calculate difference to column at right        
        sa =  sa_gst[:,[i+2,i+1]]
        ct =  ct_gst[:,[i+2,i+1]]
        z  = int_gst[:,[i+2,i+1]]

        r, a, b = gsw.rho_alpha_beta(sa.mean(axis=1),
                                     ct.mean(axis=1),
                                      z.mean(axis=1))
        ndd_r = r * (b * d(sa) - a * d(ct))

        ndc[:,i] = ndd_r + ndd_l
        
    return ndc

## Example

Let's take a look at these neutral density differences `ndc`, on the uniform grid 50 level grid remapped from the source data. First we show the original potential density field from the remapped data.

In [ ]:
n = 50
# uniform thickness from 0 to topo with n layers
h = np.linspace(0, 1, n).reshape(-1,1) * 2 * topo / n
s, t = remap(h)

plt.pcolormesh(gsw.rho(s, t, 2000))
plt.gca().invert_yaxis()

Now we actually calculate the curvature and print some statistics to see what kind of numbers we're dealing with.

In [ ]:
z = np.concatenate((np.zeros((1, h.shape[1])), h.cumsum(axis=0)), axis=0)
ndc = ndc_int(z, s, t)

print('min:\t\t{}\nmax:\t\t{}\nmean:\t\t{}\nmean (abs):\t{}\nsd:\t\t{}\nsd (abs):\t{}'.format(
    ndc.min(), ndc.max(), ndc.mean(), np.abs(ndc).mean(), ndc.std(), np.abs(ndc).std()))

In [ ]:
plt.figure(figsize=(10,6))
plt.pcolormesh(lat, z[1:-1], np.log10(np.abs(ndc)))
plt.gca().invert_yaxis()
plt.colorbar()

## Diffusing interfaces

Now we define a function to compute the optimised interface positions according to the diffusivitye quation. For the moment, we've only included the surface zooming, buoyancy and background terms. This is because we don't have any velocity data from which to calculate the shear term. We could however calculate the shear term from thermal wind if required.

In [ ]:
def diffuse(z_int, c_surf, d_surf, c_n2, dt, t_grid=3600, d_rho=0.5):
    """
    Use an implicit diffusivity equation to evolve the grid defined by z_int at timestep dt.
    """
    
    # allocate new grid
    z_next = np.empty_like(z_int)
    z_lay = (z_int[1:,:] + z_int[:-1,:]) / 2
    
    # gravity used by gsw
    g = 9.7963
    
    I = z_int.shape[0] - 1 # number of layers
    A = np.zeros((3, I+1)) # diffusion system coefficients

    # iterate over columns
    for i in range(z_int.shape[1]):
        # calculate local buoyancy term at interfaces
        # from temp/salt data at the centre of layers
        n2, z_c = gsw.Nsquared(sa[:,i], ct[:,i], z_lay[:,i])
        # drho_dz term to convert to distance
        dz_r = np.maximum((n2 * 1e4) / g**2, 0)
        # diffusivity coefficient on interfaces
        k_n2_int = dz_r / d_rho
        # interpolate the diffusivity coefficient from interfaces
        # to layers, where they apply in the diffusion equation
        # we have to (linearly) extrapolate into the top and
        # bottom layers, because we don't have the buoyancy frequency at
        # the surface or the very bottom
        f = interp1d(z_c, k_n2_int, bounds_error=False, fill_value="extrapolate")
        k_n2 = f(z_lay[:,i])
        
        # determine total grid coefficient from
        # background term 1/D, where D is local depth
        # surface stretching 1/(d + d_0) for distance
        # from surface d, and factor d_0
        #
        # k_grid = D/tgrid * (c_surf*k_surf + c_n2*k_n2 + c_b*k_b)
        k_grid = (z_int[-1,i] * (c_surf / (d_surf + np.maximum(z_lay[:,i], 0)) + \
                                 c_n2 * k_n2) \
                  + (1 - c_n2 - c_surf)) / t_grid

        # fill in implicit system coefficients
        A[0,2:]     = -dt * I**2 * k_grid[1:]
        A[2,:-2]    = -dt * I**2 * k_grid[:-1]
        A[1,[0,-1]] = 1 # boundary conditions
        A[1,1:-1]   = 1 + I**2 * dt * (k_grid[1:] + k_grid[:-1])

        # solve tridiagonal system
        z_next[:,i] = solve_banded((1, 1), A, z_int[:,i])
        
    return z_next

## Neutral Density Curvature Tendency

Instead of including the adaptation to neutral density curvature into the diffusion equation, we can implement it in a separate step, where the neutral density curvature is converted to a tendency, depending on the local stratification.

In [ ]:
def ndc_tendency(z_int, sa, ct):
    """
    Calculate interfacial tendency according to
    the neutral density curvature, calculated on
    interfaces.
    """
    
    # gravity used by gsw
    g = 9.7963
    
    # calculate neutral density curvature on the interfaces
    ndc = ndc_int(z_int, sa, ct)
    
    # calculate local buoyancy frequency on interfaces to get convert from
    # density differences to an interfacial displacement
    n2 = gsw.Nsquared(sa, ct, (z_int[1:,:] + z_int[:-1,:]) / 2)[0]
    # set a minimum value of n2 so we don't divide by zero
    n2 = np.maximum(n2, 1e-10)
    dz = (g**2 * ndc) / (1e4 * n2)

    # maximum interface movement is limited by half layer thickness in
    # the direction the interface is moving
    h = np.diff(z_int, axis=0)
    h_i = np.arange(dz.shape[0])[:,np.newaxis] + (np.sign(dz) / 2 + 0.5).astype(int)
    h_j = np.tile(np.arange(h_i.shape[1])[np.newaxis,:], (h_i.shape[0], 1))
    h_dz = h[h_i,h_j]
    dz = np.sign(dz) * np.minimum(np.abs(dz), h_dz / 2)
    
    return dz

## Generation Algorithm

Now we have the generation algorithm as follows:

- move interfaces according to their isopycnal/neutral density curvature tendency (3D term)
- enforce grid regularity
  - minimum thickess of all layers
  - conservation of total column thickness
- optimise interfaces according to diffusion equation, based on e.g. buoyancy or distance from surface

In [ ]:
def generate(z, sa, ct, dt, alpha=0.5, isopycnal=False):
    """
    Return a new grid specified by interface positions
    for a current grid given by interface positions
    as well as the density variables and an optimisation
    timestep.
    
    Parameter alpha determines the amount of tendency used
    to move interfaces in the 3D step (based on neutral
    density curvature or target isopycnals).
    """
    
    z_new = z.copy()
    
    if isopycnal:
        # calculate grid tendency from target isopycnals
        dz_tend = 0 # TODO: implement
    else:
        # calculate grid tendency term from neutral density curvature
        dz_tend = ndc_tendency(z, sa, ct)
    
    # apply tendency to interior interfaces
    z_new[1:-1] += alpha * dz_tend
    
    # calculate new layer thickness and
    # enforce minimum layer thickness and positive depths
    h = np.diff(z_new, axis=0)
    # 1mm thick minimum
    h = np.maximum(h, 1e-3)
    
    # reinflate each column
    # weight layers through the water column, for example
    # to give thinner surface layers
    w = 1 # uniform weighting
    h *= w
    # use actual bottom depths
    h *= topo / np.sum(h, axis=0)
    
    # check that we didn't introduce any negative thickness layers
    if np.any(h < 0):
        print('negative thickness at iteration', k)
    
    # recalculate z from h
    z_new = np.concatenate((np.zeros((1, h.shape[1])), h.cumsum(axis=0)), axis=0)
    
    # check that the total column thickness is preserved
    if np.any(np.abs(z_new[-1,:] - topo) > 1e-10):
        print('bottom moved at iteration', k)
        print(z_new[-1,:] - topo)
    
    # optimise layers by diffusion
    return diffuse(z_new, c_surf=0.2, d_surf=100, c_n2=0.3, dt=100, t_grid=3600, d_rho=0.5)

## Testing generation algorithm

Let's test the generation algorithm in a few cases. The first is to use the neutral density curvature as a tendency term for adjusting the grid, and the second is to move the grid toward target isopycnals. In both cases, we start with a 50-layer sigma grid (i.e. uniform thickness layers, distributed between the surface and local topography).

In [ ]:
# define initial uniform grid with 50 layers
n = 50
# uniform thickness from 0 to topo with n layers
h = np.linspace(0, 1, n).reshape(-1,1) * 2 * topo / n
z = np.concatenate((np.zeros((1, h.shape[1])), h.cumsum(axis=0)), axis=0)
sa, ct = remap(h)

# save initial neutral density curvature and density to see
# how things change after iteration (see above for plots of these
# quantities)
ndc_init  = ndc_int(z, sa, ct)
rhop_init = gsw.rho(sa, ct, 2000)

### Neutral density curvature generation

First we'll use the neutral density curvature, along with the regular grid diffusion.

In [ ]:
z_ndc = z.copy()
sa_ndc = sa
ct_ndc = ct

for i in range(10):
    z_ndc = generate(z_ndc, sa_ndc, ct_ndc, dt=100)
    h_ndc = np.diff(z_ndc, axis=0)
    sa_ndc, ct_ndc = remap(h_ndc)

In [ ]:
plt.pcolormesh(h_ndc)
plt.gca().invert_yaxis()

In [ ]:
ndc_final = ndc_int(z_ndc, sa_ndc, ct_ndc)

plt.figure(figsize=(10, 6))

ax = plt.subplot(211)
plt.pcolormesh(ndc_final)
plt.colorbar()
ax.invert_yaxis()

ax = plt.subplot(212)
plt.pcolormesh(ndc_final - ndc_init)
plt.colorbar()
ax.invert_yaxis()

In [ ]:
rhop_final = gsw.rho(sa_ndc, ct_ndc, 2000)

plt.figure(figsize=(10, 6))

ax = plt.subplot(211)
plt.pcolormesh(rhop_final)
plt.colorbar()
ax.invert_yaxis()

ax = plt.subplot(212)
plt.pcolormesh(rhop_final - rhop_init)
plt.colorbar()
ax.invert_yaxis()